In [ ]:
import asyncio
import websockets

async def send_receive_data():
    async with websockets.connect('ws://localhost:8765') as websocket:
        # 发送数据给服务器
        message = "Hello, server!"
        await websocket.send(message)
        print("Sent message to server:", message)

        # 接收服务器的响应
        response = await websocket.recv()
        print("Received response from server:", response)

asyncio.get_event_loop().run_until_complete(send_receive_data())


In [1]:
import asyncio
import websockets


async def echo(websocket, path):
    async for message in websocket:
        message = "I got your message: {}".format(message)
        await websocket.send(message)


asyncio.get_event_loop().run_until_complete(websockets.serve(echo, 'localhost', 8765))
asyncio.get_event_loop().run_forever()


RuntimeError: This event loop is already running

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import websockets
import asyncio

def send_data_to_clients(data, clients):
    # 将数据转换为字符串形式
    message = str(data)
    # 向所有客户端发送数据
    for client in clients:
        asyncio.ensure_future(client.send(message))

async def handle_client(websocket, path, clients):
    # 将新连接的客户端添加到客户端列表
    clients.append(websocket)
    try:
        while True:
            # 接收客户端消息
            message = await websocket.recv()
            print("Received message from client:", message)

            # 处理消息并准备要发送给客户端的数据
            # 这里使用示例的数据处理逻辑，您可以根据自己的需求进行修改
            processed_data = message.upper()

            # 向所有客户端发送处理后的数据
            send_data_to_clients(processed_data, clients)
    finally:
        # 断开连接时从客户端列表中移除该客户端
        clients.remove(websocket)

def start_websocket_server():
    # 创建 SparkSession 对象
    spark = SparkSession.builder.appName("WebLogAnalysis").getOrCreate()

    # 创建 StreamingContext 对象
    ssc = StreamingContext(spark.sparkContext, 5)


    # 创建一个 DStream，从 TCP socket 接收数据
    lines = ssc.socketTextStream("localhost", 9999)

    # 处理数据
    processed_lines = lines.map(lambda line: line.upper())

    # 将处理后的数据发送到 WebSocket 客户端
    processed_lines.foreachRDD(lambda rdd: rdd.foreachPartition(send_data_to_clients))

    # 启动 Spark Streaming
    ssc.start()

    # 启动 WebSocket 服务器
    clients = []
    start_server = websockets.serve(lambda ws, path: handle_client(ws, path, clients), "localhost", 8765)
    asyncio.get_event_loop().run_until_complete(start_server)
    asyncio.get_event_loop().run_forever()

# 启动 WebSocket 服务器和 Spark Streaming
start_websocket_server()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/23 00:16:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/23 00:16:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/23 00:16:26 WARN ReceiverSupervisorImpl: Restarting receiver with delay 2000 ms: Error connecting to localhost:9999
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.

RuntimeError: This event loop is already running

23/05/23 00:16:28 WARN ReceiverSupervisorImpl: Restarting receiver with delay 2000 ms: Error connecting to localhost:9999
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:613)
	at java.net.Socket.connect(Socket.java:561)
	at java.net.Socket.<init>(Socket.java:457)
	at java.net.Socket.<init>(Socket.java:234)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$re

23/05/23 00:16:34 WARN ReceiverSupervisorImpl: Restarting receiver with delay 2000 ms: Error connecting to localhost:9999
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:613)
	at java.net.Socket.connect(Socket.java:561)
	at java.net.Socket.<init>(Socket.java:457)
	at java.net.Socket.<init>(Socket.java:234)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$re

23/05/23 00:16:41 WARN ReceiverSupervisorImpl: Restarting receiver with delay 2000 ms: Error connecting to localhost:9999
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:613)
	at java.net.Socket.connect(Socket.java:561)
	at java.net.Socket.<init>(Socket.java:457)
	at java.net.Socket.<init>(Socket.java:234)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$re

23/05/23 00:16:47 WARN ReceiverSupervisorImpl: Restarting receiver with delay 2000 ms: Error connecting to localhost:9999
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:613)
	at java.net.Socket.connect(Socket.java:561)
	at java.net.Socket.<init>(Socket.java:457)
	at java.net.Socket.<init>(Socket.java:234)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$re

In [ ]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
import socket




# 创建 SparkSession
spark = SparkSession.builder.appName("SocketStreamingExample").getOrCreate()

# 创建 StreamingContext，设置批处理间隔时间
ssc = StreamingContext(spark.sparkContext, batchDuration=5)
# 创建一个输入数据流，指定监听的主机和端口号
lines = ssc.socketTextStream("localhost", 9999)
# 对输入数据流进行转换和处理
# 例如，可以使用 map、flatMap、filter 等操作来处理每个批次的数据
words = lines.flatMap(lambda line: line.split(" "))
wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
# 输出处理结果
# 例如，可以打印每个批次的结果
wordCounts.pprint()
# 启动 StreamingContext
ssc.start()
# 等待 StreamingContext 结束
ssc.awaitTermination()



23/05/23 00:19:43 WARN Utils: Your hostname, Qingyans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.3 instead (on interface en0)
23/05/23 00:19:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/23 00:19:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/23 00:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


-------------------------------------------
Time: 2023-05-23 00:19:50
-------------------------------------------



-------------------------------------------
Time: 2023-05-23 00:19:55
-------------------------------------------



23/05/23 00:19:56 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/23 00:19:56 WARN BlockManager: Block input-0-1684772396400 replicated to only 0 peer(s) instead of 1 peers


-------------------------------------------
Time: 2023-05-23 00:20:00
-------------------------------------------
('wdnmsdnxdas', 1)
('asd', 1)
('a', 1)
('das', 1)



...

-------------------------------------------
Time: 2023-05-23 00:23:15
-------------------------------------------



-------------------------------------------
Time: 2023-05-23 00:23:20
-------------------------------------------



-------------------------------------------
Time: 2023-05-23 00:23:25
-------------------------------------------



-------------------------------------------
Time: 2023-05-23 00:23:30
-------------------------------------------



-------------------------------------------
Time: 2023-05-23 00:23:35
-------------------------------------------



In [ ]:
ssc.stop()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
import socket




# 创建 SparkSession
spark = SparkSession.builder.appName("SocketStreamingExample").getOrCreate()

# 创建 StreamingContext，设置批处理间隔时间
ssc = StreamingContext(spark.sparkContext, batchDuration=5)
# 创建一个输入数据流，指定监听的主机和端口号
lines = ssc.socketTextStream("localhost", 9999)

# 定义结束作业的函数
def stop_streaming():
    ssc.stop()
    print("Streaming job stopped.")

    # 对输入数据流进行转换和处理
# 例如，可以使用 map、flatMap、filter 等操作来处理每个批次的数据
words = lines.flatMap(lambda line: line.split(" "))
wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
# # 在 DStream 上应用处理逻辑
# lines.foreachRDD(lambda rdd: rdd.foreach(print))

# 在另一个线程中监听用户输入
import threading
def read_input():
    while True:
        user_input = input()
        if user_input.lower() == "quit":
            stop_streaming()
            break

input_thread = threading.Thread(target=read_input)
input_thread.start()
wordCounts.pprint()
# 启动 Spark Streaming
ssc.start()
ssc.awaitTermination()


23/05/25 16:05:58 WARN Utils: Your hostname, Qingyans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.1.10.230 instead (on interface en0)
23/05/25 16:05:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/25 16:06:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/25 16:06:30 WARN ReceiverSupervisorImpl: Restarting receiver with delay 2000 ms: Error connecting to localhost:9999
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSock

-------------------------------------------
Time: 2023-05-25 16:06:35
-------------------------------------------



23/05/25 16:06:36 WARN ReceiverSupervisorImpl: Restarting receiver with delay 2000 ms: Error connecting to localhost:9999
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:613)
	at java.net.Socket.connect(Socket.java:561)
	at java.net.Socket.<init>(Socket.java:457)
	at java.net.Socket.<init>(Socket.java:234)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$re

-------------------------------------------
Time: 2023-05-25 16:06:40
-------------------------------------------



23/05/25 16:06:40 WARN ReceiverSupervisorImpl: Restarting receiver with delay 2000 ms: Error connecting to localhost:9999
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:613)
	at java.net.Socket.connect(Socket.java:561)
	at java.net.Socket.<init>(Socket.java:457)
	at java.net.Socket.<init>(Socket.java:234)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$re

...

23/05/25 16:25:31 WARN ReceiverSupervisorImpl: Restarting receiver with delay 2000 ms: Error connecting to localhost:9999
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:613)
	at java.net.Socket.connect(Socket.java:561)
	at java.net.Socket.<init>(Socket.java:457)
	at java.net.Socket.<init>(Socket.java:234)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$re

-------------------------------------------
Time: 2023-05-25 16:25:35
-------------------------------------------



23/05/25 16:25:35 WARN ReceiverSupervisorImpl: Restarting receiver with delay 2000 ms: Error connecting to localhost:9999
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:613)
	at java.net.Socket.connect(Socket.java:561)
	at java.net.Socket.<init>(Socket.java:457)
	at java.net.Socket.<init>(Socket.java:234)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$re

-------------------------------------------
Time: 2023-05-25 16:25:40
-------------------------------------------



23/05/25 16:25:41 WARN ReceiverSupervisorImpl: Restarting receiver with delay 2000 ms: Error connecting to localhost:9999
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:613)
	at java.net.Socket.connect(Socket.java:561)
	at java.net.Socket.<init>(Socket.java:457)
	at java.net.Socket.<init>(Socket.java:234)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$re

In [5]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
import socket
import json
import json
import websocket


# 创建 SparkSession
spark = SparkSession.builder.appName("SocketStreamingExample").getOrCreate()

# 创建 StreamingContext，设置批处理间隔时间
ssc = StreamingContext(spark.sparkContext, batchDuration=5)
# 创建一个输入数据流，指定监听的主机和端口号
lines = ssc.socketTextStream("localhost", 9999)


# 创建 WebSocket 连接
ws = websocket.WebSocket()
ws.connect("ws://localhost:8765/ws")  # 替换为实际的 WebSocket 地址


# 定义结束作业的函数
def stop_streaming():
    ssc.stop()
    print("Streaming job stopped.")


# 在另一个线程中监听用户输入
import threading
def read_input():
    while True:
        user_input = input()
        if user_input.lower() == "quit":
            stop_streaming()
            break

# 在每个批次中计算行数
def process_batch(batch):
    # 获取当前批次的行数
    count = batch.count()
    # 打印行数
    print("访问量:", count)
    # 将数据转换为 JSON 字符串
#     json_data = json.dumps(rdd.collect())
    # 发送数据给 WebSocket
    ws.send(count)
    
    

    # 处理数据的函数
def process_data(rdd):
    # 将数据转换为 JSON 字符串
    json_data = json.dumps(rdd.collect())
    
    # 发送数据给 WebSocket
    ws.send(json_data)
    
# 在 DStream 上应用 process_batch 函数
lines.foreachRDD(process_batch)
# 启动 Spark Streaming
ssc.start()
ssc.awaitTermination()

Py4JJavaError: An error occurred while calling o1679.start.
: java.lang.IllegalStateException: Only one StreamingContext may be started in this JVM. Currently running StreamingContext was started atorg.apache.spark.streaming.api.java.JavaStreamingContext.start(JavaStreamingContext.scala:557)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflect.Method.invoke(Method.java:498)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:282)
py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
py4j.commands.CallCommand.execute(CallCommand.java:79)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
py4j.ClientServerConnection.run(ClientServerConnection.java:106)
ja

访问量: 0


访问量: 0


访问量: 0


访问量: 0


...

访问量: 0


访问量: 0


访问量: 0


访问量: 0


访问量: 0


In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import json
import websocket

# 初始化 SparkContext 和 StreamingContext
sc = SparkContext(appName="WebSocketStreaming")
ssc = StreamingContext(sc, 1)

# 创建 WebSocket 连接
ws = websocket.WebSocket()
ws.connect("ws://localhost:8000/ws")  # 替换为实际的 WebSocket 地址

# 处理数据的函数
def process_data(rdd):
    # 将数据转换为 JSON 字符串
    json_data = json.dumps(rdd.collect())
    
    # 发送数据给 WebSocket
    ws.send(json_data)

# 创建一个 DStream 并处理数据
lines = ssc.socketTextStream("localhost", 9999)
lines.foreachRDD(process_data)

# 启动 StreamingContext
ssc.start()
ssc.awaitTermination()
